## BagChal modeling

The board of baghchal looks like this...

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Empty_Alquerque_board.svg/300px-Empty_Alquerque_board.svg.png" />

There are 25 vertices, where we place 4 tiger pieces and 20 goat pieces. These pieces can move if those vertices are connected by edges as shown in figure above. Initially all goat pieces are outside the board and all four tiger pieces are in four corners.

We can model this board into python as list. Example: 

`board = ['T','','','','T','','G','','','','G','G','','','','','','','','','T','','','','T']`

To print the board object in pretty form, we can use `printState(state)` method.

In [1]:
import numpy as np

In [2]:
def printState(state):
    boardStr = ""
    for i in range(0, 25, 5):
        boardStr += '---'.join(['o' if s == '' else s for s in state[i:i + 5]]) + "\n"
        if i < 20:
            boardStr += "| \ | / | \ | / |\n"  if i % 2 == 0 else "| / | \ | / | \ |\n"
    return boardStr

In [3]:
board = ['T','','','','T','','','G','G','G','','','','G','','','G','','','','T','','','G','T']
print(printState(board))

T---o---o---o---T
| \ | / | \ | / |
o---o---G---G---G
| / | \ | / | \ |
o---o---o---G---o
| \ | / | \ | / |
o---G---o---o---o
| / | \ | / | \ |
T---o---o---G---T



The board state is represented as 1D list but for computation it's easy with 2D list. For that we have utilities for converting to/from 1D to 2D.

In [4]:
def to2D(idx):
    if type(idx) is list:
        return [(int(i / 5), i % 5) for i in idx]
    else:
        return (int(idx / 5), idx % 5)


def to1D(coord):
    if type(coord) is list:
        return [i * 5 + j for (i, j) in coord]
    else:
        return coord[0] * 5 + coord[1]

Let's find the position of goats in the board...

In [5]:
idx = [i for i, x in enumerate(board) if x == 'G']
print("idx in 1D list:", idx)
print("2D repr:", to2D(idx))

idx in 1D list: [7, 8, 9, 13, 16, 23]
2D repr: [(1, 2), (1, 3), (1, 4), (2, 3), (3, 1), (4, 3)]


In [6]:
def positionOf(player, state):
    idx = [i for i, x in enumerate(state) if x == player]
    return to2D(idx)

In [7]:
print(positionOf('G', board))

[(1, 2), (1, 3), (1, 4), (2, 3), (3, 1), (4, 3)]


Moves are represented by tuples as `(source, dest)`, where source, dest are indices in 1D list representation of board. If we want to bring in goat piece from outside of board to inside source is -1.

For computing the available moves from current board state, we have these utility methods...

In [8]:
def action(position):
    moves = ['up', 'down', 'right', 'left']
    row,col = position
    if row == 0:
        moves.remove('up')
    if row == 4:
        moves.remove('down')
    if col == 0:
        moves.remove('left')
    if col == 4:
        moves.remove('right')

    return moves

# Takes current board (5X5 matrix), position to compute side moves.
def side_move(board,position):
    new_position = []
    idx = np.where(board == '')
    blank_position = set(list(zip(idx[0],idx[1])))
    moves = action(position)
    row, col = position
    # print(position)
    # Compute new position based on moves
    for move in moves:
        if move == 'up':
            new_position.append((row-1,col))
        if move == 'down':
            new_position.append((row+1,col))
        if move == 'left':
            new_position.append((row,col-1))
        if move == 'right':
            new_position.append((row,col+1))

    # Check for diagonal moves.
    if (row+col) % 2 == 0:
        if 'down' in moves:
            if 'right' in moves:
                new_position.append((row+1,col+1))
            if 'left' in moves:
                new_position.append((row+1, col-1))
        if 'up' in moves:
            if 'right' in moves:
                new_position.append((row-1, col+1))
            if 'left' in moves:
                new_position.append((row-1, col-1))
    # Return all positions which is empty (no other animal already present).
    valid_moves = list(set(new_position) & blank_position)
    # print("Valid moves:{}".format(valid_moves))
    return valid_moves

# board is 5X5 matrix, position of tiger which needs to jump.
def jumping_move(board, position):
    valid_moves = []
    new_position = []
    idx = np.where(board == '')
    blank_position = set(list(zip(idx[0],idx[1])))
    goat_idx = np.where(board == 'G')
    goat_position = set(list(zip(goat_idx[0],goat_idx[1])))
    moves = action(position)
    jump_moves = moves.copy()
    row,col = position
    for move in moves:
        if (row+2 >= 5 and move == 'down') or (row-2 <= -1 and move == 'up') or (col+2 >= 5 and move == 'right') \
                or (col-2 <= -1 and move == 'left'):
            jump_moves.remove(move)
            continue
        else:
            if move == 'up':
                new_position.append((row-2,col))
            if move == 'down':
                new_position.append((row+2,col))
            if move == 'left':
                new_position.append((row,col-2))
            if move == 'right':
                new_position.append((row,col+2))
    # print(position, jump_moves)
    # Check for diagonal moves.
    if (row+col) % 2 == 0:
        if 'down' in jump_moves:
            if 'right' in jump_moves:
                new_position.append((row+2,col+2))
            if 'left' in jump_moves:
                new_position.append((row+2, col-2))
        if 'up' in jump_moves:
            if 'right' in jump_moves:
                new_position.append((row-2, col+2))
            if 'left' in jump_moves:
                new_position.append((row-2, col-2))

    # print("Jumpables:{}".format(jumpables))
    # Check if there is a goat in-between the jump position and current position.
    for pos in set(new_position) & blank_position:
        r, c = pos
        if ((row+r)/2, (col+c)/2) in goat_position:
            valid_moves.append(pos)
        else:
            continue
    # print("Valid moves:{}".format(valid_moves))
    return valid_moves

def output_pair(data):
    output = []
    for key, values in data.items():
        src = to1D(key)
        for val in values:
            output.append((src,to1D(val)))
    return output

To compute valid goat moves...

In [9]:
def valid_goat_move(board):
    initial_moves = []
    current_board = np.array(board).reshape(5,5)
    idx = np.where(current_board == '')
    goat_data = dict()
    num_goat_in_board = len(positionOf('G', board))
    num_goats = num_goat_in_board + goat_lost
    if num_goats < 20:
        for val in (list(zip(idx[0],idx[1]))):
            initial_moves.append(to1D(val))
        return [(-1,c) for c in initial_moves]
    else:
        goat_idx = np.where(current_board == 'G')
        positions = list(zip(goat_idx[0], goat_idx[1]))
        # Compute all legal moves for a given position
        for position in positions:
            side_move = side_move(current_board, position)
            if side_move:
                goat_data[position] = side_move
    # print(goat_data)
    return output_pair(goat_data)

In [10]:
board = ['T','G','G','G','T','','','G','G','G','','T','','G','','G','G','','','','T','','','G','']
print(printState(board))

T---G---G---G---T
| \ | / | \ | / |
o---o---G---G---G
| / | \ | / | \ |
o---T---o---G---o
| \ | / | \ | / |
G---G---o---o---o
| / | \ | / | \ |
T---o---o---G---o



In [11]:
goat_lost = 1
valid_goat_move(board)

[(-1, 5),
 (-1, 6),
 (-1, 10),
 (-1, 12),
 (-1, 14),
 (-1, 17),
 (-1, 18),
 (-1, 19),
 (-1, 21),
 (-1, 22),
 (-1, 24)]

To compute valid tiger moves..

In [12]:
def valid_tiger_move(board):
    current_board = np.array(board).reshape(5,5)
    tiger_idx = np.where(current_board == 'T')
    positions = list(zip(tiger_idx[0], tiger_idx[1]))
    tiger_data = dict()
    # Check side move
    for position in positions:
        # Check slide move
        side_moves = side_move(current_board,position)
        if side_moves:
            tiger_data[position]= side_moves
        # Check jump move
        jump_moves = jumping_move(current_board,position)
        if jump_moves:
            if tiger_data.get(position):
                tiger_data[position].extend(jump_moves)
            else:
                tiger_data[position]= jump_moves
    # print(tiger_data)
    return output_pair(tiger_data)

In [13]:
valid_tiger_move(board)

[(0, 5),
 (0, 6),
 (4, 14),
 (4, 12),
 (11, 10),
 (11, 6),
 (11, 12),
 (11, 21),
 (20, 21),
 (20, 10),
 (20, 12)]

These methods are wrapped in `GameBoard` class. 

In [14]:
from baghchal import GameBoard

In [15]:
game = GameBoard()
print(game)

T---o---o---o---T
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---T



First move is made by goat player. So the available moves for current board are:

In [16]:
game.availableMoves()

[(-1, 1),
 (-1, 2),
 (-1, 3),
 (-1, 5),
 (-1, 6),
 (-1, 7),
 (-1, 8),
 (-1, 9),
 (-1, 10),
 (-1, 11),
 (-1, 12),
 (-1, 13),
 (-1, 14),
 (-1, 15),
 (-1, 16),
 (-1, 17),
 (-1, 18),
 (-1, 19),
 (-1, 21),
 (-1, 22),
 (-1, 23)]

In [17]:
game.makeMove((-1,1))
print(game)

T---G---o---o---T
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---T



Now it's turn for tiger player after goat player made its move..

In [18]:
game.availableMoves()

[(0, 5),
 (0, 6),
 (0, 2),
 (4, 3),
 (4, 8),
 (4, 9),
 (20, 15),
 (20, 16),
 (20, 21),
 (24, 19),
 (24, 18),
 (24, 23)]

In [19]:
game.makeMove((0,5))
print(game)

o---G---o---o---T
| \ | / | \ | / |
T---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---T



In [20]:
game.unmakeMove((0,5))
print(game)

T---G---o---o---T
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---T



In [21]:
game.isOver()

False

## Evaluation of board state

The evaluation functions for evaluating different state of the game are in `Evaluation` class. Different informations that we can extract from the board are here..

### Distance among goats

Distance between two goats is like manhattan distance but considering the diagonal lines in board. Then distance among goats is defined as sum of distances from one goat to every other goats divided by total number of goats in board.

In [22]:
def distBetweenGoat(pos1, pos2):
    diff0 = abs(pos2[0]-pos1[0])
    diff1 = abs(pos2[1]-pos1[1])
    diff = max(diff0, diff1)
    return diff + (1 if (diff !=0 and (pos1[0]+pos1[1])%2!=0 and (pos2[0]+pos2[1])%2!=0) else 0)


def goatDistanceAmong(game):
    sum = 0
    goatPos = game.positionOf('G', game.board)
    for pos1 in goatPos:
        for pos2 in goatPos:
            sum += distBetweenGoat(pos1,pos2)
    return sum/(len(goatPos)+1)

In [23]:
game.board = ['T','G','G','G','T','','','G','G','G','','T','','G','','G','G','','','','T','','','G','']
print(game)
goatDistanceAmong(game)

T---G---G---G---T
| \ | / | \ | / |
o---o---G---G---G
| / | \ | / | \ |
o---T---o---G---o
| \ | / | \ | / |
G---G---o---o---o
| / | \ | / | \ |
T---o---o---G---o



21.636363636363637

In [24]:
game.board = ['T','G','G','G','T','G','G','G','G','G','G','T','','G','','','','','','','T','','','','']
print(game)
goatDistanceAmong(game)

T---G---G---G---T
| \ | / | \ | / |
G---G---G---G---G
| / | \ | / | \ |
G---T---o---G---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---o



17.636363636363637

### Tiger Movability

Number of available moves for tiger in given board state...

In [25]:
def tigerMovability(game):
    return len(game.valid_tiger_move())

In [26]:
tigerMovability(game)

8

### Vulnerable Goats

Total number of goats which could be killed in current board state.

In [27]:
def vulnerableGoats(game):
    deadGoatsNextLevel = 0
    moves = game.valid_tiger_move()
    for move in moves:
        source = game.to2D(move[0])
        dest = game.to2D(move[1])
        if max(abs(dest[0]-source[0]), abs(dest[1]-source[1])) > 1:
            deadGoatsNextLevel += 1
    return deadGoatsNextLevel

In [28]:
vulnerableGoats(game)

3

### Number of goat lost

In [29]:
game.goat_lost

0

### Evaluation function

The evaluation function is linear combination of these informations that we can extract from board. So the evaluation function looks like:

$$evalFuncTiger(game) = 1*vulnerableGoats(game) + 5*game.goat\_lost + 1*tigerMovability(game) + 1*goatDistanceAmong(game)$$

For tiger player, higher values of `vulnerableGoats(game)`, `game.goat_lost`, `tigerMovability(game)` and `goatDistanceAmong(game)` is preferable. For goat player, smaller values of these individual utility functions are preferable. The total value of this evaluation function should be maximized for tiger player and minimized for goat player. In terms of goat player, same evaluation function can be used by taking negative as:
$$evalFuncGoat(game) = -1 * evalFuncTiger(game)$$

Sample of one evaluation function where goat player plays random move and tiger player uses heuristic evaluation function is like below:

In [30]:
from random import randint
def evaluate(game):
    metric = 1*vulnerableGoats(game) + 5*game.goat_lost \
        + 1*tigerMovability(game) + 1*goatDistanceAmong(game)
    if (game.player == 'T'):
        return metric
    else:
        return  randint(-10,10)

In [31]:
evaluate(game)

-6

These methods are implemented in `Evaluation` class.

In [32]:
from evaluation import Evaluation

## Minimax

In [33]:
def minimax(game, depth, maximizingPlayer, evalF):
    if depth==0 or game.isOver():
        evaluation = evalF(game)
        #print(game, "Player", game.player, "LAD", game.playerLookAhead, "eval", evaluation)
        return evaluation, None

    bestMove = None
    if maximizingPlayer==game.playerLookAhead:
        bestValue = float("-inf")
        for move in game.availableMoves():
            game.makeMove(move)
            v, _ = minimax(game, depth-1, maximizingPlayer, evalF)
            if v > bestValue:
                bestValue = v
                bestMove = move
            game.unmakeMove(move)
    else:
        bestValue = float("+inf")
        for move in game.availableMoves():
            game.makeMove(move)
            v, _ = minimax(game, depth-1, maximizingPlayer, evalF)
            if v < bestValue:
                bestValue = v
                bestMove = move
            game.unmakeMove(move)

    return bestValue, bestMove

In [34]:
game.movesExplored = 0
_, move = minimax(game, 3, 'G', Evaluation.evaluate)
move, game.getMovesExplored()

((4, 12), 904)

## Minimax with alpha beta pruning

In [35]:
def alphabeta(game, depth, maximizingPlayer, evalF, alpha=float("-inf"), beta=float("inf")):
    if depth==0 or game.isOver():
        evaluation = evalF(game)
        return evaluation, None

    bestMove = None
    if maximizingPlayer==game.playerLookAhead:
        bestValue = float("-inf")
        for move in game.availableMoves():
            game.makeMove(move)
            v, _ = alphabeta(game, depth-1, maximizingPlayer, evalF, alpha, beta)
            if v > bestValue:
                bestValue = v
                bestMove = move
            game.unmakeMove(move)
            alpha = max(bestValue, alpha)
            if alpha >= beta:
                break
    else:
        bestValue = float("+inf")
        for move in game.availableMoves():
            game.makeMove(move)
            v, _ = alphabeta(game, depth-1, maximizingPlayer, evalF, alpha, beta)
            if v < bestValue:
                bestValue = v
                bestMove = move
            game.unmakeMove(move)
            beta = min(bestValue, beta)
            if alpha >= beta:
                break

    return bestValue, bestMove

In [36]:
game.movesExplored = 0
_, move = alphabeta(game, 3, 'G', Evaluation.evaluate)
move, game.getMovesExplored()

((4, 12), 171)

## Game play

In [37]:
def gamePlay():
    game = GameBoard()
    moves = 0
    while not game.isOver() and moves<101:
        value, move = alphabeta(game, 3, game.player, evaluate)
        if move is None:
            print('move is None. Stopping')
            break
        game.makeMove(move)
        moves += 1
        print("\nPlayer", game.player, "to", move, "for value", value)
        print(game)
        game.changePlayer()
    print("Moves Explored", game.getMovesExplored(), "Moves: ", moves)
    print("EBF: ", Evaluation.ebf(game.getMovesExplored(), moves))

In [38]:
gamePlay()


Player G to (-1, 1) for value 9
T---G---o---o---T
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---T


Player T to (0, 2) for value 23.0
o---o---T---o---T
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---T
Goats Dead: 1


Player G to (-1, 22) for value 9
o---o---T---o---T
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---G---o---T
Goats Dead: 1


Player T to (2, 6) for value 26.666666666666668
o---o---o---o---T
| \ | / | \ | / |
o---T---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---G---o---T
Goats Dead: 1


Player G to (-1, 9) for value 8
o---o---o---o---T
| \ | / | \ | / |
o---T---o---o---G
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o-